In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from math import log
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400
# show more dataframe rows
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_rows', 100)

def g(w, k):
    return 1/(w+k) * ((w+k+(w-1))//w)

def read(file):
    with open(f'../data/{file}', 'r') as f:
        data = json.load(f)
    return pd.json_normalize(data)

def normalize(df):
    # Rename column 'tp.minimizer_type' to 'Minimizer type'
    df = df.rename(columns={'tp.minimizer_type': 'Minimizer type'})
    # In type column replace minizer value with random minimizer.
    df['Minimizer type'] = df['Minimizer type'].replace('Minimizer', 'Random minimizer')
    df['Minimizer type'] = df['Minimizer type'].replace('LrMinimizer', 'LR-minimizer')
    df['Minimizer type'] = df['Minimizer type'].replace('ModMinimizer', 'Mod-minimizer')
    df['Minimizer type'] = df['Minimizer type'].replace('MiniceptionNew', 'Modified miniception')


    df = df[df.k >= np.log(df.w)/np.log(df.sigma)]
    if 'tp.k0' in df.columns:
        df['param'] = df['tp.k0'].fillna(0) #+ df['tp.r'].fillna(0)

    # df.sort_values(by='Minimizer type', inplace=True)

    return df


In [ ]:
def plot(data):
    if isinstance(data, str):
        data = read(data)
    df = normalize(data)
    s = df['sigma'].unique()[0]
    ws = df.w.unique()

    for w in ws:
        plt.axhline(y=(1)/(w), color='black', linewidth=0.5)
        plt.axhline(y=(2)/(w+1), color='black', linewidth=0.5)
        ks = range(df.k.min(), df.k.max())
        plt.plot(ks, [g(w,k) for k in ks], color='red', linewidth=0.5)

    sns.lineplot(x='k', y='density', hue='Minimizer type', size='w', sizes=(1,2), data=df, legend='full', marker='.', dashes=False);
    plt.title(f'Minimizer density on random text (alphabet size σ={s}; length=5M)')
    plt.xlabel('Kmer length k')
    plt.ylabel('Density')
    plt.yscale('log', base=2)
    plt.yticks([2/(w+1) for w in ws]+[1/w for w in ws],[f'{2/(w+1):.3f}' for w in ws]+[f'{1/w:.3f}' for w in df.w.unique()])
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')

    plt.savefig(f'../fig/density_{s}.svg', bbox_inches='tight')
    plt.show()

In [ ]:
plot('density_4.json')

In [ ]:
def plot(file):
    df = read(file)
    s = df['sigma'].unique()[0]

    # Plot position distribution.
    if True:
        # Flatten the data json object and add a new dist column.
        ddf = df.explode('positions').reset_index().rename(columns={'index' : 'position'})
        ddf['position'] = ddf.groupby('position').cumcount()
        g = sns.relplot(ddf,
                        kind='line',
                        x='position',
                        y='positions',
                        hue='Minimizer type',
                        row="k",
                        col="w",


                        facet_kws={
                            'sharex': "col",
                            'sharey': True,
                            'margin_titles': True,
                            'ylim': (0.7, 1.3),
                        },
                        height=2.5,

                        )
        g.fig.subplots_adjust(top=0.92) # adjust the Figure in rp
        g.fig.suptitle('Minimizer position distribution')
        g.set_axis_labels("Selected position", "Relative frequency")
        plt.savefig(f'../fig/positions_{s}.svg', bbox_inches='tight')
        plt.show()

    # Plot distance distribution.
    if True:
        # Flatten the data json object and add a new dist column.
        ddf = df.explode('dists').reset_index().rename(columns={'index' : 'dist'})
        ddf['dist'] = ddf.groupby('dist').cumcount()-ddf.w
        ddf = ddf[(ddf.dists > 0) | (ddf.dist >= 0)]
        # display(ddf[ddf.dist<0])
        g = sns.relplot(ddf,
                        kind='line',
                        x='dist',
                        y='dists',
                        hue='Minimizer type',
                        row="k",
                        col="w",
                        facet_kws={
                            'sharex': "col",
                            'sharey': "row",
                            'margin_titles': True,
                            # 'ylim': (0.7, 1.3),
                        },
                        height=2.5,
                        )
        g.fig.subplots_adjust(top=0.92) # adjust the Figure in rp
        g.fig.suptitle('Minimizer distance distribution')
        g.set_axis_labels("Anchor distance", "Relative frequency")
        plt.savefig(f'../fig/dists_{s}.svg', bbox_inches='tight')
        plt.show()

    # Plot parameter choice.
    if False:
        # Show pivot table of optimal parameter for rows k and cols w
        pdf = df[df["Minimizer type"]=="Modified miniception"].pivot_table(index='k', columns='w', values='param')
        display(pdf)

plot('stats_256.json')

In [ ]:
%load_ext autoreload
%autoreload 2
import header as h
from header import plt
from sympy import Symbol, Lambda, Max, Min
from matplotlib.ticker import MaxNLocator

n = 10000000
w = 24
ks = range(4, 80)
ws = range(2, 21)
sigma = 4
h.gen(n, sigma)

def plot(name, tps, add=0, bold_last=False, plot_w=False, ymin=0.050, ymax=0.082, rol=True, df=False, **kwargs):
    global w
    data = []
    if plot_w:
        title = f'Densities and lower bounds (σ={sigma})'
    else:
        title = f'Densities and lower bounds (σ={sigma}, w={w}, t={t})'
    ax = plt.gca()
    h.style(ax, w, ks, title=title, plot_w=plot_w, df=df)
    ax.yaxis.set_major_locator(MaxNLocator(nbins=10))
    ax.grid(True, axis="y", color="#ccc", linewidth=0.5)
    ax.set_ylim(ymin=ymin, ymax=ymax)


    if plot_w:
        wks = [(w, 1) for w in ws]
        xs = ws
    else:
        wks = [(w, k) for k in ks]
        xs = ks

    for (i, tp_args) in enumerate(tps):
        if isinstance(tp_args, tuple):
            (tp, args) = tp_args
        else:
            tp = tp_args
            args = {}
        
        last = 1.5 if bold_last and i == len(tps)-1 else 0

        print(f'{tp}: {args}', flush=True)
        ds = []
        for (w, k) in wks:
            my_args = {k:v for (k,v) in args.items() }
            if 'lc' in my_args:
                my_args.pop('lc')
            if 'ls' in my_args:
                my_args.pop('ls')
            if 'o' in args:
                my_args['offset'] = args['o'](k)
            if 'k0' in args and not isinstance(args['k0'], int):
                my_args['k0'] = my_args['k0'](k)
            d = h.density(tp, w, k, sigma, **my_args)
            if df:
                d *= w+1
            my_args['k'] = k
            my_args['w'] = w
            my_args['d'] = d
            my_args['tp'] = tp
            my_args['sigma'] = sigma

            data.append(my_args)
            ds.append(d)
        print(ds[0])
        lc = args.get('lc', None)
        ls = args.get('ls', None)
        label = str(args.get('label', tp))
        if label is None:
            args.pop('label')
            if 'lc' in args:
                args.pop('lc')
            if 'ls' in args:
                args.pop('ls')
            label = f'{tp}: {args}'
        if not args.get('ao', False) and not args.get('aot', False):
            ls = 'solid'
        if args.get('ao', False) and not args.get('aot', False):
            label += ' ak'
            if not ls:
                ls = 'dotted'
        if not args.get('ao', False) and args.get('aot', False):
            label += ' at'
            if not ls:
                ls = 'dashed'
        if args.get('ao', False) and args.get('aot', False):
            label += ' at,ak'
            if not ls:
                ls = 'dashdot'
        lw = 1+last
        if args.get('modulo', False):
            lw -= 0.5
        
        line, = plt.plot(xs, ds, label=None, linestyle=ls, color=lc, marker='o', markersize=2+last, lw=0)

        # Rolling minimum of ds
        dm = 1
        dms = []
        for d in ds:
            if rol:
                dm = min(dm, d)
            else:
                dm = d
            dms.append(dm)
        plt.plot(xs, dms, label=None, color=line.get_color(), markersize=0, lw=lw)
        # Add unified legend item.
        plt.plot([], [], label=label, color=line.get_color(), marker='o', markersize=2+last, lw=lw)


    for _ in range(add):
        plt.plot([], [], label=' ', alpha=0)

    h.plot_lower_bounds(sigma, xs, wks, df=df, **kwargs)

    if plot_w:
        loc = 'upper center'
    else:
        loc = 'lower center'
    plt.legend(loc=loc, bbox_to_anchor=(0, 0.03, 1, 1), ncols=4, mode='expand', fontsize=6)
    plt.savefig(f'{name}.png', bbox_inches='tight', dpi=400)
    plt.savefig(f'{name}.svg', bbox_inches='tight')
    plt.close()
    return data

In [ ]:
# MAGIC playground
n = 100000
def tps_t():
    global t
    k = Symbol('k')
    return [
        #'AltRot', 'Random', 'AntiLex', 'Decycling',
        #'DoubleDecycling',
        #('BdAnchor', {'r': t}),
        #'SusAnchorLex', 'SusAnchorALex',
        #('Random', {'r': t, 'mod': 1}),
        #('Random', {'r': t, 'mod': 1, 't': 1}),
        #('Random', {'r': t, 'mod': 1, 't': 1, 'o1': 0, 'o2': 1}),
        #('Random', {'r': t, 'mod': 1, 't': 1, 'o1': 1, 'o2': 0}),
        #('Random', {'r': t, 'mod': 1, 't': 1, 'o1': 1, 'o2': 1}),
        #('AntiLex', {'r': t, 'mod': 1}),
        #('Decycling', {'r': t, 'mod': 0}),
        ('DoubleDecycling', {'r': t, 'mod': 0}),
        #('BdAnchor', {'r': t, 'mod': 1}),
        #('SusAnchorLex', {'r': t, 'mod': 1}),
        ('SusAnchorALex', {'r': 1, 'mod': 1}),
        #('OpenClosed', {'r': t, 'closed': 1, 'label': 'C' }),
        #('OpenClosed', {'r': t, 'open': 1, 'label': 'O' }),
        #('OpenClosed', {'r': t, 'open': 1, 'open_tmer': 1, 'label': 'Ot' }),
        # ('OpenClosed', {'r': t, 'open': 1, 'open_tmer': 1, 'label': 'OtA', 'anti_tmer': 1 }),
        ('OpenClosed', {'r': t, 'open': 1, 'closed': 1, 'label': 'OC' }),

        #('OpenClosed', {'r': t, 'mod': 1, 'open': 1, 'closed': 0, 'label': 'OM' }),
        #('OpenClosed', {'r': t, 'mod': 1, 'open': 0, 'closed': 1, 'label': 'CM' }),
        #('OpenClosed', {'r': t, 'open': 1, 'closed': 1, 'label': 'OCa', 'anti_lex': 1 }),
        #('OpenClosed', {'r': t, 'mod': 1, 'open': 1, 'closed': 1, 'label': 'OCMa', 'anti_lex': 1 }),
        #('OpenClosed', {'r': t, 'mod': 1, 'open': 1, 'closed': 0, 'label': 'OMa', 'anti_lex': 1 }),
        #('OpenClosed', {'r': t, 'mod': 1, 'open': 0, 'closed': 1, 'label': 'CMa', 'anti_lex': 1 }),
        # ('OpenClosed', {'r': t, 'mod': 1, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCMt' }),
        # ('OpenClosed', {'r': t, 'mod': 1, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCMtA', 'anti_tmer': 1 }),
        #('OpenClosed', {'r': t, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCtA', 'anti_tmer': 1, 'lc': 'black' }),
        # ('OpenClosed', {'r': t, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCtk'}),
        #('OpenClosed', {'r': t, 'open_tmer': 1, 'open': 1, 'closed': 1, 'closed_tmer': 1, 'label': 'OCt'}),
        #('OpenClosed', {'r': t, 'modulo': 1, 'open': 1, 'closed': 1, 'label': 'OCM ext' }),
        #('OpenClosed', {'r': t, 'modulo': 1, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCM(t) ext' }),
        #('Threshold', {'r': 1, 'th': 170, 'label': 'Threshold 1', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        #('Threshold', {'r': 1, 'th': 180, 'label': 'Threshold 1', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        #('Threshold', {'r': 1, 'th': 196, 'label': 'Threshold 1', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        #('Threshold', {'r': 1, 'th': 200, 'label': 'Threshold 1', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        #('Threshold', {'r': 1, 'th': 210, 'label': 'Threshold 2', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        ('Threshold', {'r': 1, 'th': 220, 'label': 'Threshold 2', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        ('Threshold', {'r': 1, 'th': 230, 'label': 'Threshold 3', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        ('Threshold', {'r': 1, 'th': 235, 'label': 'Threshold 3', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        ('Threshold', {'r': 1, 'th': 240, 'label': 'Threshold 4', 'open': 0, 'h': 1, 'loose': 1, 'mod': 0}),
        #('Threshold', {'r': 1, 'th': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1}),
        #('Threshold', {'r': t, 'th': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 't': 1, 'o1': 0}),
        #('Threshold', {'r': 1, 'th': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 't': 1, 'o2': 1}),
        #('Threshold', {'r': 1, 'th': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 't': 1, 'o1': 0, 'o2': 1}),
        #('Threshold', {'r': t, 'th': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 't': 2, 'o1': 0}),
        #('Threshold', {'r': 1, 'th': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 't': 2, 'o2': 1}),
        #('Threshold', {'r': 1, 'th': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 't': 2, 'o1': 0, 'o2': 1}),
        # ('OpenClosed', {'r': t, 'closed': 1, 'closed_tmer': 1, 'label': 'Ct' }),
    ]

import os
os.environ["RUST_BACKTRACE"] = "1"
w = 96
ks = range(1, 30)
sigma = 4
t=1
sigma = 256
t=1
h.gen(n, sigma)
tps=tps_t()
if False:
    plt.plot(
        ks,
        [oc_mod_mini_density(w, k, k, t, open=False) for k in ks],
        color='red',
        linewidth=3,
        label="C (formula)",
        alpha=0.3,
    )
    print('A')
    plt.plot(
        ks,
        [oc_mod_mini_density(w, k, t+(k-t)%w, t, open=False) for k in ks],
        color='red',
        linewidth=3,
        label="CM (formula)",
        alpha=0.5,
    )
    print('B')
    plt.plot(
        ks,
        [oc_mod_mini_density(w, k, k, t, closed=False) for k in ks],
        color='red',
        linewidth=3,
        label="O (formula)",
        alpha=0.3,
    )
    print('C')
    plt.plot(
        ks,
        [oc_mod_mini_density(w, k, t+(k-t)%w, t, closed=False) for k in ks],
        color='red',
        linewidth=3,
        label="OM (formula)",
        alpha=0.5,
    )
    print('D')
    plt.plot(
        ks,
        [oc_mod_mini_density(w, k, k, t) for k in ks],
        color='red',
        linewidth=3,
        label="OC (formula)",
        alpha=0.3,
    )
    print('E')
    plt.plot(
        ks,
        [oc_mod_mini_density(w, k, t+(k-t)%w, t) for k in ks],
        color='red',
        linewidth=3,
        label="OCM (formula)",
        alpha=0.5,
    )
    print('F')
if False:
    plt.plot(
        ks,
        [2/(w + (k-t+2)/2) for k in ks],
        color='black',
        linewidth=1,
        label="k<<w approx",
        alpha=1,
    )
    plt.plot(
        ks,
        [(2*w+k-t)/w/(w+(k+1-t)) for k in ks],
        color='red',
        linewidth=1,
        label="ctn",
        alpha=1,
    )

plot('plot', tps, loose=False, tight=True, bold_last = False, ymin=None, ymax=None, rol=False, df=False);


1.9984599830598135
DoubleDecycling: {'r': 1, 'mod': 0}


1.9984599830598135
OpenClosed: {'r': 1, 'open': 1, 'label': 'O'}


1.9984599830598135
OpenClosed: {'r': 1, 'open': 1, 'closed': 1, 'label': 'OC'}


1.9984599830598135
OpenClosed: {'r': 4, 'mod': 1, 'open': 0, 'closed': 0, 'label': 'M4'}


1.9984599830598135


In [ ]:
           
def oc_mod_mini_density(w, k, t, s, open=True, closed=True):
    d = syncmer_count(w, t, s, 0, w+k, open, closed)
    p_charged = 0
    for (o, c, co, cc), p in d.items():
        if open and o > 0:
            p_charged += p * co / o
        elif closed and c > 0:
            p_charged += p * cc / c
        else:
            p_charged += p * ((2+(k-t+w-1)//w) / (w+k-t+1))
    return p_charged

In [ ]:
# Open-Closed-mini density analysis corresponding to the paper, breaking ties by kmer hash
from functools import cache
from collections import defaultdict

# Return values:
# dict mapping (#open, #closed, #charged open, #charged closed) -> probability
# When open>0, closed=0 since they don't matter anymore.
# Probabilities must always sum to 1.

def add_distributions(a, b, tup):
    (o, c, oc, cc) = tup
    # Default dict with value 0.
    d = defaultdict(int)
    for ((o1, c1, oc1, cc1), p1) in a.items():
        for ((o2, c2, oc2, cc2), p2) in b.items():
            if o+o1+o2 > 0:
                t = (o+o1+o2, 0, oc+oc1+oc2, 0)
            else:
                t = (o+o1+o2, c+c1+c2, oc+oc1+oc2, cc+cc1+cc2)
            d[t] += p1*p2
    return d

def charged(w, i):
    return int((i%w)==0)

# Syncmers of length t, with every w'th syncmer in the remaining window l..r charged.
# returns: probability of count of open/closed (charged) syncmers.
@cache
def syncmer_count(w, t, s, l, r, open, closed):
    if r - l < t:
        return {(0, 0, 0, 0): 1}

    d = defaultdict(int)

    for i in range(l, r-s+1):
        L = syncmer_count(w, t, s, l, i+s-1, open, closed)
        R = syncmer_count(w, t, s, i+1, r, open, closed)

        (o, c, oc, cc) = (0,0,0,0)

        if open and l <= i-(t-s)//2 <= r-t:
            # Open syncmer.
            o += 1
            oc += charged(w, i-(t-s)//2)
        if closed and i <= r-t:
            c += 1
            cc += charged(w, i)
        if closed and l <= i+s-t:
            c += 1
            cc += charged(w, i+s-t)
        di = add_distributions(L, R, (o, c, oc, cc))

        for k, v in di.items():
            d[k] += v / (r-l)
    return d
        

In [ ]:
# Open-Closed-mini density analysis, break ties by tmer hash
from functools import cache

# Situation:
# ..x..xx.......y..y..yy.  w chars in total.
#        <----->           gap
# ** **  ******* ** **  *  free
# <--                      l
#                     -->  r
#                     
# Return probability of a shared open syncmer for the sigma=infinite case.
@cache
def _open_closed_density_t(w, k, gap, free, l, r, c_edge=None):
    assert free >= gap
    dl = k//2
    dr = (k-1)//2

    # No open syncmer.
    if gap < k:
        # But maybe the largest closed syncmer is shared.
        if not c_edge: return 1
        return 0

    p = 0

    # Choose one of the 'irrelevant' slots.
    if free > gap:
        p += (free-gap) * _open_closed_density_t(w,k,gap,free-1, l, r, c_edge) / free
    else:
        p += 0

    # Count and probability of having a non-shared open kmer.
    new_open_cnt = int(l) + int(r)
    p += 0
    # Count and probability of having a shared open kmer.
    shared_open_cnt = gap - k + 1 - new_open_cnt
    p += shared_open_cnt/free

    # The smallest closed syncmer is always created in the first iteration.
    first = gap == w

    # Next smallest element is at position dl from the left, and does not make an open syncmer.
    for i in range(dl):
        c_edge = i==0 or gap-i-1 == k-1 if first else c_edge
        p += _open_closed_density_t(w, k, gap-i-1, free-1, False, r, c_edge) / free

    # Next smallest element is at position dl from the left, and does not make an open syncmer.
    for i in range(dr):
        c_edge = i==0 or gap-i-1 == k-1 if first else c_edge
        p += _open_closed_density_t(w, k, gap-i-1, free-1, l, False, c_edge) / free
    
    return p
        
            
def open_closed_density_t(w, k, r):
    k2 = max(k+1-r, 1)
    c = w+k2
    d = _open_closed_density_t(c, k2, c, c, True, True)
    # print(w, k, d)
    return 1-d


In [ ]:
# Open-Closed-mini-anti density analysis, break ties by min tmer and max tmer
from functools import cache

# Situation:
# ..x..xx.......y..y..yy.  w chars in total.
#        <----->           gap
# ** **  ******* ** **  *  free
# <--                      l
#                     -->  r
#                     
# Return probability of a shared open syncmer for the sigma=infinite case.
@cache
def _open_closed_density_a(w, k, gap, free, l, r, l_edge=None, r_edge=None):
    assert free >= gap
    dl = k//2
    dr = (k-1)//2

    # No open syncmer.
    if gap < k:
        # But maybe the largest closed syncmer is shared.
        if not l_edge and not r_edge: return 1
        return 0

    p = 0

    # Choose one of the 'irrelevant' slots.
    if free > gap:
        p += (free-gap) * _open_closed_density_a(w,k,gap,free-1, l, r, l_edge, r_edge) / free
    else:
        p += 0

    # Count and probability of having a non-shared open kmer.
    new_open_cnt = int(l) + int(r)
    p += 0
    # Count and probability of having a shared open kmer.
    shared_open_cnt = gap - k + 1 - new_open_cnt
    p += shared_open_cnt/free

    # Next smallest element is at position dl from the left, and does not make an open syncmer.
    for i in range(dl):
        if gap-i-1 < k-1:
            # Small gap remaining, so no closed syncmer is created.
            p += _open_closed_density_a(w, k, gap-i-1, free-1, False, r, l_edge, r_edge) / free
        elif gap-i-1 == k-1:
            # This is the largest closed syncmer.
            p += int(not r) / free
            pass
        else:
            # Create a closed syncmer.
            p += _open_closed_density_a(w, k, gap-i-1, free-1, False, r, l and i==0, False) / free

    # Next smallest element is at position dl from the left, and does not make an open syncmer.
    for i in range(dr):
        if gap-i-1 < k-1:
            # Small gap remaining, so no closed syncmer is created.
            p += _open_closed_density_a(w, k, gap-i-1, free-1, l, False, l_edge, r_edge) / free
        elif gap-i-1 == k-1:
            # This is the largest closed syncmer.
            p += int(not l) / free
            pass
        else:
            # Create a closed syncmer.
            p += _open_closed_density_a(w, k, gap-i-1, free-1, l, False, False, r and i==0) / free
    
    return p
        
            
def open_closed_density_a(w, k, r):
    k2 = max(k+1-r, 1)
    c = w+k2
    d = _open_closed_density_a(c, k2, c, c, True, True)
    # print(w, k, d)
    return 1-d


In [ ]:
# Open-mini density analysis
from functools import cache

# Situation:
# ..x..xx.......y..y..yy.  w chars in total.
#        <----->           gap
# ** **  ******* ** **  *  free
# <--                      l
#                     -->  r
#                     
# Return probability of a shared open syncmer for the sigma=infinite case.
@cache
def _open_density(w, k, gap, free, l, r):
    assert free >= gap
    dl = k//2
    dr = (k-1)//2
    # First step
    if l and r:
        assert gap == w
        assert free == w
        # Minimum in 'middle' w-1-k positions -> open syncmer.
        # .--********--. 8 = w-k-1
        assert w-k-1 >= 0
        shared_open = (w-k-1)/free
        cases = w-k-1
        # --*--....--*-- k=5 w=14
        new_open = 0 * 2/free
        cases += 2
        # One of the d positions left or right:
        recurse = 0
        for i in range(dl):
            recurse += _open_density(w, k, gap-i-1, free-1, False, True) / free
            cases += 1
        for i in range(dr):
            recurse += _open_density(w, k, gap-i-1, free-1, True, False) / free
            cases += 1
        assert cases == free
        return shared_open + new_open + recurse
    if not l and not r:
        # Open syncmer not possible anymore.
        if gap < k:
            return 0
        # Minimum in 'middle' gap+1-k positions -> open syncmer.
        # <-------> gap=9
        # --*****-- 5 = gap-k+1
        assert gap-k+1 >= 0
        shared_open = (gap-k+1)/free
        cases = gap-k+1
        # Choose one of the 'irrelevant' slots.
        if free > gap:
            recurse = (free-gap) * _open_density(w,k,gap,free-1, False, False) / free
        else:
            recurse = 0
        cases += free - gap
        # Choose one of the slots close to the edge of the gap.
        for i in range(dl):
            recurse += _open_density(w, k, gap-i-1, free-1, False, False) / free
            cases += 1
        for i in range(dr):
            recurse += _open_density(w, k, gap-i-1, free-1, False, False) / free
            cases += 1
        assert cases == free
        return shared_open + recurse
    if l or r:
        # Open syncmer not possible anymore.
        if gap < k:
            return 0
        # Minimum in the single edge position: non-shared open syncmer.
        new_open = 0 * 1 / free
        cases = 1
        # Minimum in 'middle' gap+1-k positions, but not at one edge -> shared open syncmer.
        # <-------> gap=9
        # .--****-- 4 = gap-k
        assert gap-k >= 0
        shared_open = (gap-k)/free
        cases += gap-k
        # Choose one of the 'irrelevant' slots.
        if free > gap:
            recurse = (free-gap) * _open_density(w,k,gap,free-1, True, False) / free
        else:
            recurse = 0
        cases += free - gap
        # Choose one of the slots close to the edge of the gap..
        # .. on the 'edge' side:
        for i in range(dl if l else dr):
            recurse += _open_density(w, k, gap-i-1, free-1, False, False) / free
            cases += 1
        # .. on the 'inner' side:
        for i in range(dr if l else dl):
            recurse += _open_density(w, k, gap-i-1, free-1, True, False) / free
            cases += 1
        assert cases == free, f"w {w} k {k} free {free} gap {gap} cases {cases} dl {dl} dr {dr}"
        return shared_open + recurse
    assert False
        
            
def open_density(w, k, r):
    k2 = max(k+1-r, 1)
    c = w+k2
    d = _open_density(c, k2, c, c, True, True)
    # print(w, k, d)
    return 1-d




In [ ]:
# Position & Distance analysis
w = 24
n = 10000000
sigma = 256
h.gen(n, sigma)

mat = []

def analyze(out):
    (d, p, dst, t) = out
    print('density', d)
    print('pos:  ', end='')
    for x in p:
        print(f'{x:.4f} ', end='')
    print()
    print('dist: ', end='')
    for x in dst[w+1:]:
        print(f'{x:.4f} ', end='')
    print()
    print('transfer:')
    global mat
    mat = t
    for (i, x) in enumerate(t):
        print(f'{i:2} -> ', end='')
        for y in x:
            print(f'{y:.5f} ', end='')
        print()

k = 6
print('target ', 2/(w+k))
# analyze(h.minimizers.stats('Minimizer', h._text, w, k, sigma, t=1))
# analyze(h.minimizers.stats('Minimizer', h._text, w, k+1, sigma, t=2))
# analyze(h.minimizers.stats('OpenSyncmerMinimizer', h._text, w, k, sigma, t=1))
# analyze(h.minimizers.stats('OpenSyncmerMinimizer', h._text, w, k+1, sigma, t=2))
#analyze(h.minimizers.stats('ClosedSyncmerMinimizer', h._text, w, k+1, sigma, t=208, open=0, h=1, loose=0))
#analyze(h.minimizers.stats('ClosedSyncmerMinimizer', h._text, w, k+1, sigma, t=208, open=1, h=1, loose=0))
# analyze(h.minimizers.stats('OpenSyncmerMinimizer', h._text, w, k+2, sigma, t=3))
# analyze(h.minimizers.stats('SusAnchor', h._text, w, 1, sigma, ao=0))
# analyze(h.minimizers.stats('SusAnchor', h._text, w, 1, sigma, ao=1))

analyze(h.minimizers.stats('OpenClosed', h._text, 24, 5, 256, r=t, open=1))


In [ ]:
import numpy as np
from numpy.linalg import matrix_power
mate = [
    [1,1,1,1,2],
    [5,0,0,0,1],
    [0,5,0,0,1],
    [0,0,5,0,1],
    [0,0,0,5,1],
]
m = np.array(mat)
row_sums = m.sum(axis=1)
m_norm = m / row_sums[:, np.newaxis]
display(m_norm)
display(m_norm.sum(axis=0))
display(m_norm.sum(axis=1))
m_pow = matrix_power(m_norm, 8)
display(m_pow)
display(m_pow.sum(axis=0))
display(m_pow.sum(axis=1))


In [ ]:
# Cycle stats
w = 5
n = 100000000
sigma = 256
h.gen(n, sigma)

def analyze(out):
    (d, p) = out
    print('density', d)
    print('cnt:  ', end='')
    for x in p:
        print(f'{x:.5f} ', end='')
    print(flush=True)

k = 3
print('k=1    ', 2/(w+1))
print('avg    ', 2/(w+(k+1)/2))
print('target ', 2/(w+k))
print('hyp    ', (2+1/(w-1))/(w+k))
for l in range(k+w-1,k+w+1):
    print(l)
    # analyze(h.minimizers.cycle_stats('OpenSyncmerMinimizer', h._text, w, k, l, sigma, t=1))
    analyze(h.minimizers.cycle_stats('OpenSyncmerMinimizer', h._text, w, k+1, l, sigma, t=2))
    # analyze(h.minimizers.cycle_stats('OpenSyncmerMinimizer', h._text, w, k+2, l, sigma, t=3))


In [ ]:
# Playground for selection schemes
    
tps = [
    # ('BdAnchor', {'r': 0, 'label': None, 'lc': '#00ee00', 'label': 'Bd-anchor r=0'}),
    # ('BdAnchor', {'r': 1, 'label': None, 'lc': '#00cc00', 'label': 'Bd-anchor r=1'}),
    # ('BdAnchor', {'r': 2, 'label': None, 'lc': '#009900', 'label': 'Bd-anchor r=2'}),
    # ('BdAnchor', {'r': 3, 'label': None, 'lc': '#006600', 'label': 'Bd-anchor r=3'}),
    ('SusAnchor', {'lc': '#0000aa'}),
    ('SusAnchor', {'ao': 1, 'lc': '#0000ff'}),
]

sigma = 2
h.gen(n, sigma)
plot('selection', tps, plot_w=True, loose=False, tight=True, bold_last=True);


In [ ]:
      #'AltRot', 'Random', 'AntiLex', 'Decycling',
        #'DoubleDecycling',
        #('BdAnchor', {'r': t}),
        #'SusAnchorLex', 'SusAnchorALex',
        ('Random', {'r': t, 'mod': 1}),
        #('AntiLex', {'r': t, 'mod': 1}),
        ('DoubleDecycling', {'r': t, 'mod': 1}),
        #('BdAnchor', {'r': t, 'mod': 1}),
        #('SusAnchorLex', {'r': t, 'mod': 1}),
        #('SusAnchorALex', {'mod': 1}),
        ('OpenClosed', {'r': t, 'open': 1, 'label': 'O' }),
        ('OpenClosed', {'r': t, 'open': 1, 'open_tmer': 1, 'label': 'Ot' }),
        ## ('OpenClosed', {'r': t, 'open': 1, 'open_tmer': 1, 'label': 'OtA', 'anti_tmer': 1 }),
        # ('OpenClosed', {'r': t, 'mod': 1, 'open': 1, 'closed': 1, 'label': 'OCM' }),
        # ('OpenClosed', {'r': t, 'mod': 1, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCMt' }),
        # ('OpenClosed', {'r': t, 'mod': 1, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCMtA', 'anti_tmer': 1 }),
        ('OpenClosed', {'r': t, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCtA', 'anti_tmer': 1, 'lc': 'black' }),
        ('OpenClosed', {'r': t, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCtk'}),
        ('OpenClosed', {'r': t, 'open_tmer': 1, 'open': 1, 'closed': 1, 'closed_tmer': 1, 'label': 'OCt'}),
        #('OpenClosed', {'r': t, 'modulo': 1, 'open': 1, 'closed': 1, 'label': 'OCM ext' }),
        #('OpenClosed', {'r': t, 'modulo': 1, 'open_tmer': 1, 'open': 1, 'closed': 1, 'label': 'OCM(t) ext' }),
        #('Threshold', {'r': t, 't': 3, 'label': 'Threshold', 'open': 0, 'h': 1, 'loose': 1, 'mod': 1}),
        # ('OpenClosed', {'r': t, 'closed': 1, 'label': 'C' }),
        # ('OpenClosed', {'r': t, 'closed': 1, 'closed_tmer': 1, 'label': 'Ct' }),

In [ ]:
# Plot sampling functions



def tps_t():
    global t
    k = Symbol('k')
    return [
        ('Random', {'lc': 'orange', 'label': 'Random minimizer'}),
        ('DoubleDecycling', {'lc': 'black', 'label': 'Double Decycling'}),
        ('OpenClosed', {'r': t, 'closed': 1, 'lc': '#00dd00', 'label': 'C=Miniception'}),
        ('OpenClosed', {'r': t, 'mod': 1, 'lc': 'blue', 'label': 'Mod-mini'}),
        ('OpenClosed', {'r': t, 'open': 1, 'lc': '#bb0066', 'label': 'O'}),
        ('OpenClosed', {'r': t, 'open': 1, 'closed': 1, 'lc': '#6600bb', 'label': 'OC'}),
        ('OpenClosed', {'r': t, 'mod': 1, 'open': 1, 'closed': 1, 'open_tmer': 0, 'lc': 'magenta',  'label': 'OC-Mod-mini'}),
        ('OpenClosed', {'r': t, 'o': lambda k: ((((k-t-2)%w)+2)//2)%w,  'modulo': 1, 'open': 1, 'closed': 1, 'open_tmer': 1, 'lc': 'purple',  'label': 'OC mod-offset by tmer'}),
        ('SusAnchorALex', {'r': t, 'mod': 1, 'lc': 'brown', 'label': 'AntiLexSus-Mod'}),
        ('Threshold', {'r': t, 't': 3, 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 'lc': 'teal', 'label': 'Threshold'}),
        ('Threshold', {'r': t, 't': 196, 'open': 0, 'h': 1, 'loose': 1, 'mod': 1, 'lc': 'teal', 'label': 'Threshold'}),
    ]
n=1000000
ks = range(1, 75)
sigma = 256
h.gen(n, sigma)
t=1
tps=tps_t()
plot('blog/1a-random', tps[:1], add=10, loose=False, bold_last=True);
plot('blog/1b-decycling', tps[:2], add=9, loose=False, bold_last=True);
plot('blog/1c-miniception', tps[:3], add=8, loose=False, bold_last=True);
plot('blog/2-modmini', tps[:4], add=7, loose=False, bold_last=True);
plot('blog/3-lower-bound', tps[:4], add=6);
plot('blog/4-open-syncmer', tps[:5], add=5, bold_last = True);
plot('blog/5-open-closed-syncmer', tps[:6], add=4, bold_last = True);
plot('blog/6-oc-mod-mini', tps[:7], add=3, bold_last = True);
plot('blog/7-oc-mod-offset', tps[:8], add=2, bold_last = True);

sigma = 4
h.gen(n, sigma)
t=4
tps=tps_t()

plot('blog/8-anti-lex', tps[:9], add=1, bold_last = True);
plot('blog/9-threshold', tps[:10], add=0, bold_last = True);

sigma = 256
h.gen(n, sigma)
t=1
tps=tps_t()
plot('blog/10-s256', tps[:9]+tps[10:], add=0, bold_last = False);


In [ ]:
# Plot selection functions
    
tps = [
    ('BdAnchor', {'r': 0, 'label': None, 'lc': '#00ee00', 'label': 'Bd-anchor r=0'}),
    ('BdAnchor', {'r': 1, 'label': None, 'lc': '#00cc00', 'label': 'Bd-anchor r=1'}),
    ('BdAnchor', {'r': 2, 'label': None, 'lc': '#009900', 'label': 'Bd-anchor r=2'}),
    ('BdAnchor', {'r': 3, 'label': None, 'lc': '#006600', 'label': 'Bd-anchor r=3'}),
    ('SmallestUniq', {'scramble': 0, 'xor': 1, 'prefix': 0, 'label': None, 'lc': '#0000aa', 'label': 'Sus-anchor'}),
    ('SmallestUniq', {'scramble': 3, 'xor': 1, 'prefix': 0, 'label': None, 'lc': '#0000ff', 'label': 'Sus-anchor scrambled'}),
    ('SmallestUniq', {'scramble': 1, 'xor': 1, 'prefix': 0, 'label': None, 'lc': '#0000ff', 'label': 'Sus-anchor scrambled'}),
]

sigma = 4
h.gen(n, sigma)
plot('blog/20-bd-anchors', tps[:4], plot_w=True, add=3);
plot('blog/21-sus-anchors', tps[:5], plot_w=True, add=2, bold_last=True);
plot('blog/22-scramble', tps[:6], plot_w=True, add=1, tight=False, loose=True, bold_last=True);
plot('blog/23-scramble', tps[:6], plot_w=True, add=1, tight=True, loose=False, bold_last=True);
sigma = 3
h.gen(n, sigma)
plot('blog/24-s3', tps[:6], plot_w=True, add=1, tight=True, loose=False, bold_last=True);
sigma = 2
h.gen(n, sigma)
plot('blog/25-s2', tps[:5]+tps[-1:], plot_w=True, add=1, tight=True, loose=False, bold_last=True);


In [ ]:
# New closed syncmer tryout
w = 24
k = 6
sigma = 256
threshold = (1-1/k)*sigma
print('threshold: ', threshold)
n = 1000
print('gen..')
h.gen(n, sigma)
def is_good_l(kmer):
    if kmer[0] < threshold: return 0
    for i in range(1, len(kmer)):
        if kmer[i] >= threshold:
            return i
    return len(kmer)
    # return kmer[0] >= threshold and all(c < threshold for c in kmer[1:])
def is_good_r(kmer):
    if kmer[-1] < threshold: return 0
    for i in range(len(kmer)-2, -1, -1):
        if kmer[i] >= threshold:
            return len(kmer) - 1 - i
    return len(kmer)
    # return kmer[0] >= threshold and all(c < threshold for c in kmer[1:])
def closed_hash(kmer):
    l = is_good_l(kmer)
    return (-l, kmer)
    r = is_good_r(kmer)
    return (-max(l,r), kmer)
    if r == len(kmer):
        return (-r, kmer)
    return (-l, kmer)

def closed_syncmer(w, k, window):
    assert len(window) == w+k-1
    best = (closed_hash(window[0:k]), 0)
    for i in range(1, w):
        best = min(best, (closed_hash(window[i:i+k]), i))
    return (best[1], best[0])


In [ ]:
count = 0
print('range..')
for i in range(n-k):
    g = is_good(h._text[i:i+k])
    if g:
        count += 1
        # print(i)
print(count)

In [ ]:
def antilex(kmer, prefer_short=False):
    s = max(16 - len(kmer), 0)
    prefix = [0 if prefer_short else 3]*16
    mask = 0
    for i in range(16-s):
        prefix[15-i] = kmer[i] ^ mask
        # if kmer[i] == kmer[0]:
        #     mask ^= 3
    prefix[15] ^= 3
    val = 0
    for i in range(16):
        val *= 4
        val += prefix[15-i]
    rep = False
    rep = rep or (len(kmer) >= 4 and (kmer[0] != kmer[1]) and (kmer[0:2] == kmer[2:4]))
    rep = rep or (len(kmer) >= 6 and (kmer[0] != kmer[1]) and (kmer[0:3] == kmer[3:6]))
    # rep = rep or (len(kmer) >= 5 and (kmer[0] != kmer[1]) and (kmer[0:2] == kmer[3:5]))
    return (rep, val)

def sus_anchor(w, k, lmer):
    if lmer == [1,0,1,0,1,0]:
        return (4, (None, [1,0]))
    best = (antilex2(lmer), 0)
    for i in range(1, w):
        best = min(best, (antilex2(lmer[i:]), i))
    return (best[1], best[0])

def free(kmer):
    l = len(kmer)
    #if l < 3:
    #    return False
    # if l >= 2 and kmer[0] == kmer[1]:
    #     return False
    #if l == 3 and kmer[0] == kmer[2]:
    #    return False
    #if l >= 4 and kmer[0] != kmer[1] and kmer[0:2] == kmer[2:4]:
    #    return False
    #if l >= 5 and kmer[0] != kmer[1] and kmer[0:2] == kmer[3:5]:
    #    return False
    # if l > 2 and kmer[0:2] == kmer[l-2:l]:
    #     return False
    # if l > 3 and kmer[0:3] == kmer[l-3:l]:
    #     return False
    # if l > 4 and kmer[0:4] == kmer[l-4:l]:
    #     return False
    return True

def antilex2(kmer, last=9):
    return (not free(kmer), [-kmer[0]] + kmer[1:] + [last])

# with threshold
def antilex3(kmer, last=9):
    return (not free(kmer), [int(kmer[0] < threshold)] + kmer[1:] + [last])

def sus_anchor2(w, k, lmer):
    assert len(lmer) == w+k-1
    vals = []
    for start in range(w):
        for end in range(start+1, len(lmer)+1):
            kmer = lmer[start:end]
        
            vals.append((antilex2(kmer), start))
    vals.sort()
    for i in range(len(vals)):
        if i > 0 and vals[i][0] == vals[i-1][0]:
            continue
        if i +1<len(vals) and vals[i][0] == vals[i+1][0]:
            continue
        return (vals[i][1], vals[i][0])
    print('only dups')
    return (vals[0][1], vals[0][0])

def susmod(w, k, lmer):
    if k == w+1:
        pos, val = sus_anchor2(w+w, k-w, lmer)
        return (pos % w, val)
    if k == w:
        pos, val = sus_anchor2(w+w-1, k-w+1, lmer)
        return (pos % (w-0), val)
    assert False
    

def cyclic_count(w,k,lmer, f, ll = None):
    if ll is None:
        ll = w+k
    lmer2 = lmer +  lmer[:-1]

    ps = {}
    count = {}
    for i in range(ll):
        (pos, val) = f(w, k, lmer2[i:i+w+k-1])
        # print('A:', lmer2[i:i+w+k-1], pos, val)
        # (pos2, val2) = sus_anchor2(w, 1, lmer2[i:i+w])
        # print('B:', lmer2[i:i+w], pos2, val2)
        x = (i + pos)%(ll)
        ps[x] = (val, i)
        if x not in count:
            count[x] = 0
        count[x] += 1
        
    if len(ps) != K:
        for p,(v, i) in sorted(ps.items()):
            # print('            ', ' '*p, *lmer2[p:p+w+k], ' '*(w+k-p), f'{v} {i}' , sep='')
            s = [lmer2[p+i] for i in range(w+k)]
            c = count[p]
            print('            ', ' '*(2*p), ','.join(str(x) for x in s), ' '*2*(ll-p), f'{c} {','.join(str(x) for x in v[1])} {i}' , sep='')

    # ps = {}
    # for i in range(w+1):
    #     (pos, val) = sus_anchor2(w, 1, lmer2[i:i+w])
    #     ps[(i + pos)%(w+1)] = (val, i)
    # if len(ps) > 2:
    #     print('B:')
    #     for p,(v, i) in sorted(ps.items()):
    #         print('            ', ' '*p, *lmer2[p:p+w+1], ' '*(w+1-p), f'{v} {i}' , sep='')

    return len(ps)

w = 6
sigma = 2
k = 1
ll = w+k
threshold = (1-1/k)*sigma
threshold = 75

import random
x = 0
N = 1000
K = 2
for c in range(N):
    lmer = [random.randint(0, sigma-1) for _ in range(ll)]
    #lmer = [1 if c >= threshold else 0 for c in lmer]
    lmer2 = lmer +  lmer[:-1]
    f=sus_anchor
    cnt = cyclic_count(w, k, lmer, f=f, ll=ll) 
    if cnt != K:
        x += 1
        # print(f'{x:>3} {c:>6}: ', *lmer,' ', cnt, sep='')
        s = [lmer[i] for i in range(ll)]
        print(f'{x:>3} {c:>6}: ', ','.join(str(x) for x in s),' ', cnt, sep='')
        print(f'          : ', end='')
        for i in range(ll):
            print(f'{(i+f(w, k, lmer2[i:i+ll-1])[0])%ll:1},', end='')
        print()
        print()
    assert cnt >= K

d = (K + x / N) / ll
print(K/ll, 'opt')
print(d, 'real')


In [ ]:


def new(w, k, lmer):
    # 1. replace 11 by 01 from left to right
    # 2. replace 101 by 001 from left to right
    # 3. replace 1001 by 0001 from left to right
    # ...
    # until only 0 left. Then take leftmost 1 of step before.
    d = 1
    #print(lmer)
    while True:
        new_lmer = [1 if lmer[i] == 1 and (i+d >= len(lmer) or lmer[i+d] == 0) else 0 for i in range(len(lmer))]
        #print(new_lmer)
        if new_lmer[:w].count(1) == 0:
            break
        lmer = new_lmer
        if new_lmer.count(1) <= 2:
            break
        d += 1

    idx = lmer.index(1) if 1 in lmer[:w] else 0
    val = lmer[idx:]
    return (idx, (None, val))

new(9, 9, [0,0,0,0,1,1,1,1,1,1,0,1,1,0,0,1,1,0])